# method 1
from github: jalajthanaki/stock_price_prediction
<br><br>
06282020: error, can not find newsapi.articles


In [4]:
"""
About:
Python wrapper for the New York Times Archive API
https://developer.nytimes.com/article_search_v2.json
"""
from newsapi.articles import Articles
import sys,json
import requests

key = '522497f7b4b940b7946eeed6909ed817'
params = {}
api = Articles(API_KEY=key)
reload(sys)
sys.setdefaultencoding('utf8')


class APIKeyException(Exception):
    def __init__(self, message): self.message = message


class InvalidQueryException(Exception):
    def __init__(self, message): self.message = message


class ArchiveAPI(object):
    def __init__(self, key=None):
        self.key = key
        self.root = 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}'
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'Warning: API Key required. Please visit {}'
            raise APIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None, key=None, ):
        if not key: key = self.key
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1851
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
        url = self.root.format(year, month, key)
        #print url ##boting: remove error message
        r = requests.get(url)
        return r.json()




ModuleNotFoundError: No module named 'newsapi.articles'

In [5]:
import newsapi 

In [8]:
from newsapi.articles import Articles

ModuleNotFoundError: No module named 'newsapi.articles'

In [7]:
dir(newsapi)

['NewsApiClient',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'const',
 'newsapi_auth',
 'newsapi_client',
 'newsapi_exception',
 'utils']

In [ ]:
years = [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
api = ArchiveAPI('522497f7b4b940b7946eeed6909ed817')
for year in years:
    for month in months:
        mydict = api.query(year, month)
        file_str = './data/nytimes' + str(year) + '-' + '{:02}'.format(
            month) + '.json'
        with open(file_str, 'w') as fout:
            json.dump(mydict, fout)
        fout.close()

# method 2
from github: jasonyip184/StockSentimentTrading
 


In [14]:
import nltk
nltk.download('vader_lexicon')

import warnings
warnings.filterwarnings('ignore')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /Users/wangbo-
[nltk_data]     ting/nltk_data...


In [19]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import pprint

date_sentiments = {}

for i in range(1,11):
    page = urlopen('https://www.businesstimes.com.sg/search/facebook?page='+str(i)).read()
    soup = BeautifulSoup(page, features="html.parser")
    posts = soup.findAll("div", {"class": "media-body"})
    for post in posts:
        time.sleep(1)
        url = post.a['href']
        date = post.time.text
        print(date, url)
        try:
            link_page = urlopen(url).read()
        except:
            url = url[:-2]
            link_page = urlopen(url).read()
        link_soup = BeautifulSoup(link_page)
        sentences = link_soup.findAll("p")
        passage = ""
        for sentence in sentences:
            passage += sentence.text
        sentiment = sia.polarity_scores(passage)['compound']
        date_sentiments.setdefault(date, []).append(sentiment)

date_sentiment = {}

for k,v in date_sentiments.items():
    date_sentiment[datetime.strptime(k, '%d %b %Y').date() + timedelta(days=1)] = round(sum(v)/float(len(v)),3)

earliest_date = min(date_sentiment.keys())

print(date_sentiment)

29 Jun 2020 https://www.businesstimes.com.sg/technology/facebook-ad-boycott-campaign-to-go-global-organisers-say
27 Jun 2020 https://www.businesstimes.com.sg/technology/unilever-to-stop-advertising-on-facebook-twitter-and-instagram-in-us
27 Jun 2020 https://www.businesstimes.com.sg/technology/facebook-ramps-up-efforts-to-curb-hateful-content-in-ads
26 Jun 2020 https://www.businesstimes.com.sg/technology/verizon-suspends-advertising-on-facebook-joins-growing-boycott
24 Jun 2020 https://www.businesstimes.com.sg/consumer/ben-jerrys-joins-facebook-ad-boycott-over-hate-speech
23 Jun 2020 https://www.businesstimes.com.sg/government-economy/biden-campaign-asks-twitter-facebook-to-remove-trump-posts-bashing-mail-in-voting
23 Jun 2020 https://www.businesstimes.com.sg/technology/microsoft-ends-game-streaming-teams-up-with-facebook
19 Jun 2020 https://www.businesstimes.com.sg/government-economy/facebook-pulls-trump-ads-over-nazi-symbol
18 Jun 2020 https://www.businesstimes.com.sg/technology/faceb

13 Feb 2020 https://www.businesstimes.com.sg/technology/facebook-owned-whatsapp-says-it-has-2-billion-users
08 Feb 2020 https://www.businesstimes.com.sg/technology/twitter-says-facebook-messenger-accounts-hacked
31 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-to-remove-virus-misinformation-after-who-declares-global-emergency
31 Jan 2020 https://www.businesstimes.com.sg/technology/facebooks-revenue-rises-again-but-more-slowly-than-ever
30 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-results-beat-forecasts-but-shares-take-a-hit
29 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-rolls-out-tool-globally-to-clear-third-party-data
28 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-asks-employees-to-put-off-non-essential-travel-to-china
23 Jan 2020 https://www.businesstimes.com.sg/banking-finance/facebook-spurred-central-banks-to-study-digital-currencies-ex-boj-exec
22 Jan 2020 https://www.businesstimes.com.sg/banking-finance/vodaf

In [17]:
date_sentiment

18 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-must-disclose-app-records-for-probe-us-judge-rules
 Hello, your email is unverified. Please confirm  
0.9844
10 Jan 2020 https://www.businesstimes.com.sg/technology/lawmakers-facebooks-steps-to-tackle-deepfake-videos-inadequate
 Hello, your email is unverified. Please confirm  
-0.8632
08 Jan 2020 https://www.businesstimes.com.sg/consumer/facebooks-new-singapore-team-to-serve-china-advertising-business
 Hello, your email is unverified. Please confirm  
0.995
08 Jan 2020 https://www.businesstimes.com.sg/technology/facebook-tightening-policy-against-deepfakes
 Hello, your email is unverified. Please confirm  
0.9126
07 Jan 2020 https://www.businesstimes.com.sg/government-economy/trump-says-zuckerberg-told-him-hes-facebooks-number-one
 Hello, your email is unverified. Please confirm  
-0.628
28 Dec 2019 https://www.businesstimes.com.sg/banking-finance/facebooks-libra-has-failed-in-current-form-says-swiss-president
 Hello, yo

In [18]:
date_sentiment

NameError: name 'date_sentiment' is not defined

# method 3
from github dineshdaultani/StockPredictions

need an api 'key'

In [22]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 20 00:10:29 2017
@author: Dinesh
"""
# Below part is not required
########## News API ########################################################
# from newsapi import NewsAPI

# key = '96af62a035db45bda517a9ca62a25ac3'
# params = {}
# api = NewsAPI(key)
# sources = api.sources(params)
# articles = api.articles(sources[0]['id'], params)

################ NY Times API #############################################
import sys, csv, json
#reload(sys) #boting: reload has been removed since python 3.4
#sys.setdefaultencoding('utf8') #boting: sys.setdefaultencoding is not recommended for use


import requests
"""
About:
Python wrapper for the New York Times Archive API 
https://developer.nytimes.com/article_search_v2.json
"""

class APIKeyException(Exception):
    def __init__(self, message): self.message = message 

class InvalidQueryException(Exception):
    def __init__(self, message): self.message = message 

class ArchiveAPI(object):
    def __init__(self, key=None):
        """
        Initializes the ArchiveAPI class. Raises an exception if no API key is given.
        :param key: New York Times API Key
        """
        self.key = key
        self.root = 'http://api.nytimes.com/svc/archive/v1/{}/{}.json?api-key={}' 
        if not self.key:
            nyt_dev_page = 'http://developer.nytimes.com/docs/reference/keys'
            exception_str = 'Warning: API Key required. Please visit {}'
            raise NoAPIKeyException(exception_str.format(nyt_dev_page))

    def query(self, year=None, month=None, key=None,):
        """
        Calls the archive API and returns the results as a dictionary.
        :param key: Defaults to the API key used to initialize the ArchiveAPI class.
        """
        if not key: key = self.key
        if (year < 1882) or not (0 < month < 13):
            # currently the Archive API only supports year >= 1882
            exception_str = 'Invalid query: See http://developer.nytimes.com/archive_api.json'
            raise InvalidQueryException(exception_str)
        url = self.root.format(year, month, key)
        r = requests.get(url)
        return r.json()


        

    

In [26]:
import os

In [27]:
# Replace below key with your NYTimes Developer key
api = ArchiveAPI('key')

#years = [2016, 2015, 2014, 2013, 2012, 2011, 2010, 2009, 2008, 2007]
years = [2016]
months = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

for year in years:
    for month in months:
        mydict = api.query(year, month)
        #file_str = '/Users/Dinesh/Documents/Project Stock predictions/data/nytimes/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        if not os.path.exists('./news/nytimes/'):
            os.makedirs('./news/nytimes/')
        file_str = './news/nytimes/' + str(year) + '-' + '{:02}'.format(month) + '.json'
        with open(file_str, 'w') as fout:
            json.dump(mydict, fout)
        fout.close()

In [ ]:
# method 4
from github: 